In [4]:
from flask import Flask, render_template
from flask import request # to have access to the query string

In [2]:
app = Flask(__name__)

@app.route('/multiply')
@app.route('/multiply/<int:num1>/<int:num2>')
def multiply(num1=5, num2=5):
    res = 5*5
    return '{}'.format(res)


In [3]:
# templates in Flask
# long way

app = Flask(__name__)

@app.route('/multiply')
@app.route('/multiply/<int:num1>/<int:num2>')
def multiply(num1=5, num2=5):
# triple quotes keeps line breaks
    return """
    <!doctype html>
    <html>
    <head><title>Multiply!</title></head>
    <body>
    <h1>{} * {} = {}</h1>
    </body>
    </html>
    """.format(num1, num2, num1*num2)

In [ ]:
# short way - need to make a directory 
# - named templates - Flask will automatically look for it
# make name.html
# print variables - jinja2 - {{ num1 }}

@app.route('/multiply')
@app.route('/multiply/<int:num1>/<int:num2>')
def multiply(num1=5, num2=5):
    # need to pass the variables to render_template
    return render_template("name.html", num1=num1, num2=num2)

# or ...

def multiple(num1, num2):
    context = {'num1':num1, 'num2':num2}
    return render_template("name.html", **context)



In [ ]:
# template inheritance
''' put all the things that stay the same in layout.html
and make them into blocks {% block title%}{% endblock %}
{%   %} = command (as opposed to print or evaluate in jinja2)

when you use the inherited template in other .html files - 
add at the top:
{% extends "layout.html" %}
{% block title %}Howdy!{% endblock %} <-- overwrites the inherited title

if you want to add to a block & keep the inherited:
{% block title %}Howdy! | {{ super() }}{% endblock %}
{% block content %}

'''

In [ ]:
# static directory
''' this is where css goes
reference to this goes into the layout.html that is inherited by other 
name.html docs
<link rel="stylesheet" href="/static/styles.css"> <-- goes in the <head>

'''

In [ ]:
# javascript in static directory
'''
alert("Howdy");
and then need to add to layout.html:
goes in <body>:
{% block scripts %}{% endblock %}

goes where you need it to happen:
<script src="/static/scripts.js"></script> <-- or at least it did in this 
case bc that is where he wanted the thing to happen

'''

In [ ]:
from flask import redirect
from flask import url_for
'''
return redirect(url_for('index'))

'''


In [ ]:
# cookies
''' need json - import json
set cookies - set on response - the thing that goes back to the server 

import make_response from flask

-- need to do this bc you don't actually have the thing yet to send back 
-- so you have to make it!
def save():
    response = make_response(redirect(url_for('index')))
    response.set_cookie('character', json.dumps(dict(request.form.items())))
    return response


get the cookie back out:

def get_saved_data():
    try:
        data = json.loads(request.cookies.get('character'))
    except TypeError:
        data = {}
    return data

'''

In [ ]:
# looping through a list given to a template (in html)
<ul>                                     
{% for option in options %}
  <li>{{ option.name }}</li>
{% endfor %} 
</ul>

# option["name"] also works

In [ ]:
# showing selections
{% if saves.get('footwear') %}
<div class="footwear"><img src="/static/img/bear-{{ saves["footwear"] }}.svg"></div>
{% endif %}


In [ ]:
# flash messages
# in app.py, import flash from flask
# using session - and in flask those are cryptographically signed
# need to make secret key - just type random things - long is good

app = Flask(__name__)
app.secret_key = 'asdkjboeh238094tn;sdcnoad.k'

# & then update save method
# call flask method
@app.route('/save', methods=["POST"])
def save():
    flash("Alright!")
    ...

# but then need something that prints out the flash message
# layout.html - right before block content
<body>
    <div class="wrap no-bottom messages">
        {% with messages = get_flashed_messages() %}
        {% if messages %}
        <ul class="flashes">
            {% for message in messages %}
            <li> {{ message }}</li>
            {% endfor %}
        </ul>
        {% endif %}
        {% endwith %}
    </div>
    {% block content %}{% endblock %}

In [ ]:
# works with Heroku well

In [ ]:
# peewee - Python database

from peewee import *

from models import Challenge

all_challenges = Challenge.select()

Challenge.create(language= "Ruby", name= "Booleans")

sorted_challenges = Challenge.select().order_by(Challenge.steps)

In [ ]:
#!/usr/bin/env python3
import datetime
from collections import OrderedDict
import sys
import os

from peewee import *

db=SqliteDatabase('diary.db')


class Entry(Model):
    # content
    content = TextField()
    # timestamp - if you put parenthesis on .now below -it would be 
    # dt now, when you wrote it, not when you run it
    timestamp = DateTimeField(default=datetime.datetime.now)
    
    class Meta:
        database = db

def initialize():
    """Create the db and table if they don't exist"""
        db.connect()
        db.create_tables([Entry], safe=True)
        
def clear():
    # windows newer than 95 is int
    os.system('cls' if os.name =='nt' else 'clear')
        
def menu_loop():
    ''' show the menu'''
    choice = None
    
    while choice != 'q':
        clear()
        print("Enter 'q' to quit.")
        for key, value in menu.items():
            print('{}) {}'.format(key, value.__doc__))
        choice = input('Action:  ').lower().strip()
        
        if choice in menu:
            clear()
            menu[choice]()

def add_entry():
    """ Add an entry """
    print("Enter your entry, press ctrl+d when finished.")
    data = sys.stdin.read().strip()
    
    if data:
        if input('Save entry [Yn] ').lower() != 'n':
            Entry.create(content=data)
            print("Saved successfully!")
    
def view_entry(search_query=None):
    """ View all entries """
    entries = Entry.select().order_by(Entry.timestamp.desc())
    if search_query:
        entries = entries.where(Entry.content.contains(search_query))
    
    for entry in entries:
        timestamp = entry.timestamp.strftime('%A %B %d, %Y %I:%M%p')
        clear()
        print(timestamp)
        print('='*len(timestamp))
        print(entry.content)
        print('\n\n'+'='*len(timestamp))
        print('n) next entry')
        print('d) delete entry')
        print('q) return to main menu')
        
        next_action = input('Action: [Ndq]  ').lower().strip()
        if next_action =='q':
            break
        elif next_action == 'd':
            delete_entry(entry)

def search_entries():
    """ Search entries for a string."""
    view_entries(input('Search query: '))
    
def delete_entry(entry):
    """ Delete an entry. """
    if input("Are you sure? [yN]  ").lower() == "y":
        entry.delete_instance()
        print("Entry deleted!")
    
menu = OrderedDict([
    ('a', add_entry),
    ('v', view_entries),
    ('s', search_entries)
])

    
# To prevent code from being executed when you import a module, 
# put it into a function or class, or inside which if condition:
    
if __name__ == '__main__':
    initialize()
    menu_loop()

In [ ]:
# where with multiple conditions:
challenges = Challenge.select().where((Challenge.name == name) | (Challenge.language == language))


In [ ]:
# call os level stuff - import os
# os.system() lets us use system-level applications
def clear():
    # windows newer than 95 is int
    os.system('cls' if os.name =='nt' else 'clear')

In [ ]:
bio = TextField(default="") <-- empty string for its default makes it optional

In [ ]:
# !/usr/bin/env python3 <-- shebang line needed on linus/ workspaces/ mac
# if then run in command line: chmod +x diary.py, can run w/ ./diary.py

In [ ]:
# ordered dict as menu - remembers order you add it in, uses list


In [ ]:
# __doc__ --> gets docstring from function! cool

In [ ]:
# organize imports - stuff that comes from python, blank line, 3rd party stuff